In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

(optional) Exporting a Model from PyTorch to ONNX and Running it using ONNX Runtime
===================================================================================

```{=html}
<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>
```
```{=html}
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">
```
```{=html}
<p>As of PyTorch 2.1, there are two versions of ONNX Exporter.</p>
```
```{=html}
</div>
```
In this tutorial, we describe how to convert a model defined in PyTorch
into the ONNX format using the TorchScript `torch.onnx.export` ONNX
exporter.

The exported model will be executed with ONNX Runtime. ONNX Runtime is a
performance-focused engine for ONNX models, which inferences efficiently
across multiple platforms and hardware (Windows, Linux, and Mac and on
both CPUs and GPUs). ONNX Runtime has proved to considerably increase
performance over multiple models as explained
[here](https://cloudblogs.microsoft.com/opensource/2019/05/22/onnx-runtime-machine-learning-inferencing-0-4-release)

For this tutorial, you will need to install
[ONNX](https://github.com/onnx/onnx) and [ONNX
Runtime](https://github.com/microsoft/onnxruntime). You can get binary
builds of ONNX and ONNX Runtime with

``` {.bash}
%%bash
pip install onnx onnxruntime
```

ONNX Runtime recommends using the latest stable runtime for PyTorch.


In [ ]:
# Some standard imports

Super-resolution is a way of increasing the resolution of images, videos
and is widely used in image processing or video editing. For this
tutorial, we will use a small super-resolution model.

First, let\'s create a `SuperResolution` model in PyTorch. This model
uses the efficient sub-pixel convolution layer described in [\"Real-Time
Single Image and Video Super-Resolution Using an Efficient Sub-Pixel
Convolutional Neural Network\" - Shi et
al](https://arxiv.org/abs/1609.05158) for increasing the resolution of
an image by an upscale factor. The model expects the Y component of the
`YCbCr` of an image as an input, and outputs the upscaled Y component in
super resolution.

[The
model](https://github.com/pytorch/examples/blob/master/super_resolution/model.py)
comes directly from PyTorch\'s examples without modification:


In [ ]:
# Super Resolution model definition in PyTorch






























# Create the super-resolution model by using the above model definition.

Ordinarily, you would now train this model; however, for this tutorial,
we will instead download some pretrained weights. Note that this model
was not trained fully for good accuracy and is used here for
demonstration purposes only.

It is important to call `torch_model.eval()` or
`torch_model.train(False)` before exporting the model, to turn the model
to inference mode. This is required since operators like dropout or
batchnorm behave differently in inference and training mode.


In [ ]:
# Load pretrained model weights



# Initialize model with the pretrained weights





# set the model to inference mode

Exporting a model in PyTorch works via tracing or scripting. This
tutorial will use as an example a model exported by tracing. To export a
model, we call the `torch.onnx.export()` function. This will execute the
model, recording a trace of what operators are used to compute the
outputs. Because `export` runs the model, we need to provide an input
tensor `x`. The values in this can be random as long as it is the right
type and size. Note that the input size will be fixed in the exported
ONNX graph for all the input\'s dimensions, unless specified as a
dynamic axes. In this example we export the model with an input of
batch\_size 1, but then specify the first dimension as dynamic in the
`dynamic_axes` parameter in `torch.onnx.export()`. The exported model
will thus accept inputs of size \[batch\_size, 1, 224, 224\] where
batch\_size can be variable.

To learn more details about PyTorch\'s export interface, check out the
[torch.onnx documentation](https://pytorch.org/docs/master/onnx.html).


In [ ]:
# Input to the model



# Export the model

We also computed `torch_out`, the output after of the model, which we
will use to verify that the model we exported computes the same values
when run in ONNX Runtime.

But before verifying the model\'s output with ONNX Runtime, we will
check the ONNX model with ONNX API. First,
`onnx.load("super_resolution.onnx")` will load the saved model and will
output a `onnx.ModelProto` structure (a top-level file/container format
for bundling a ML model. For more information [onnx.proto
documentation](https://github.com/onnx/onnx/blob/master/onnx/onnx.proto).).
Then, `onnx.checker.check_model(onnx_model)` will verify the model\'s
structure and confirm that the model has a valid schema. The validity of
the ONNX graph is verified by checking the model\'s version, the
graph\'s structure, as well as the nodes and their inputs and outputs.


Now let\'s compute the output using ONNX Runtime\'s Python APIs. This
part can normally be done in a separate process or on another machine,
but we will continue in the same process so that we can verify that ONNX
Runtime and PyTorch are computing the same value for the network.

In order to run the model with ONNX Runtime, we need to create an
inference session for the model with the chosen configuration parameters
(here we use the default config). Once the session is created, we
evaluate the model using the run() API. The output of this call is a
list containing the outputs of the model computed by ONNX Runtime.


In [ ]:
# compute ONNX Runtime output prediction



# compare ONNX Runtime and PyTorch results

We should see that the output of PyTorch and ONNX Runtime runs match
numerically with the given precision (`rtol=1e-03` and `atol=1e-05`). As
a side-note, if they do not match then there is an issue in the ONNX
exporter, so please contact us in that case.


Timing Comparison Between Models
================================


Since ONNX models optimize for inference speed, running the same data on
an ONNX model instead of a native pytorch model should result in an
improvement of up to 2x. Improvement is more pronounced with higher
batch sizes.


Running the model on an image using ONNX Runtime
================================================


So far we have exported a model from PyTorch and shown how to load it
and run it in ONNX Runtime with a dummy tensor as an input.


For this tutorial, we will use a famous cat image used widely which
looks like below

![](https://pytorch.org/tutorials/_static/img/cat_224x224.jpg)


First, let\'s load the image, preprocess it using standard PIL python
library. Note that this preprocessing is the standard practice of
processing data for training/testing neural networks.

We first resize the image to fit the size of the model\'s input
(224x224). Then we split the image into its Y, Cb, and Cr components.
These components represent a grayscale image (Y), and the
blue-difference (Cb) and red-difference (Cr) chroma components. The Y
component being more sensitive to the human eye, we are interested in
this component which we will be transforming. After extracting the Y
component, we convert it to a tensor which will be the input of our
model.


Now, as a next step, let\'s take the tensor representing the grayscale
resized cat image and run the super-resolution model in ONNX Runtime as
explained previously.


At this point, the output of the model is a tensor. Now, we\'ll process
the output of the model to construct back the final output image from
the output tensor, and save the image. The post-processing steps have
been adopted from PyTorch implementation of super-resolution model
[here](https://github.com/pytorch/examples/blob/master/super_resolution/super_resolve.py).


In [ ]:
# get the output image follow post-processing step from PyTorch implementation







# Save the image, we will compare this with the output image from mobile device


# Save resized original image (without super-resolution)

Here is the comparison between the two images:

![](https://pytorch.org/tutorials/_static/img/cat_resized.jpg)

Low-resolution image

![](https://pytorch.org/tutorials/_static/img/cat_superres_with_ort.jpg)

Image after super-resolution

ONNX Runtime being a cross platform engine, you can run it across
multiple platforms and on both CPUs and GPUs.

ONNX Runtime can also be deployed to the cloud for model inferencing
using Azure Machine Learning Services. More information
[here](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-onnx).

More information about ONNX Runtime\'s performance
[here](https://onnxruntime.ai/docs/performance).

For more information about ONNX Runtime
[here](https://github.com/microsoft/onnxruntime).


In [ ]:
# %%%%%%RUNNABLE_CODE_REMOVED%%%%%%